In [1]:
#ProgramName: Schema Compare
#Purpose: Find differences in object details between QA and Prod ODS
#Author:  Greg Turmel, Director, Data Governance 
#Date:    2020.08.30 - 2021.06.30
#Errata:  0.1 Improvements can be made to script using for/looping through the databases

import os, sys, argparse, csv, pyodbc, sql, time, datetime
import sqlalchemy as db

from dotenv import load_dotenv # add this line
import pandas as pd
import numpy as np

load_dotenv() # add this line
user = os.getenv('MySQLeUser')
password = os.getenv('MySQLeUserPass')
host = os.getenv('MySQLeHOST')
db = os.getenv('MySQLeDB')

In [2]:
################################################################################
# Variables: 
################################################################################
now01 = datetime.datetime.now()
csvtype = 'Base' # csvtype - use 'Test or Base'
Dbug = 'Off'     # Debugging - use On or Off  
# Process files in procpath
file00 = 'dbListC.csv'
file01 = 'dbListN.csv'
file02 = 'tblListC.csv'
file03 = 'tblListN.csv'
file04 = 'schemaListC.csv'
file05 = 'schemaListN.csv'
# Report files in csvpath
file06 = 'compareReport0.csv'
file07 = 'compareReport1.csv'
file08 = 'compareReport2.csv'
file09 = 'compareReport3.csv'
file10 = 'compareReport4.csv'
file11 = 'compareReport5.csv'
file12 = 'compareReport6.csv'
file13 = 'compareReport7.csv'
file14 = 'compareReport8.csv'
file15 = 'compareReport9.csv'
file16 = 'compareReport10.csv'
file17 = 'compareReport11.csv'
file18 = 'compareReport12.csv'
file19 = 'collectionP1.csv'
file20 = 'collectionQ1.csv'

In [3]:
################################################################################
# Specify a path for reports: Make sure output location is available for write 
################################################################################
now01 = datetime.datetime.now()
import os, errno, pathlib2
try:
    os.makedirs(r"C:\Users\e201873842\Documents\Jupyter\Daily\Compare\Process", exist_ok=True)  # succeeds even if directory exists.
    os.makedirs(r"C:\Users\e201873842\Documents\Jupyter\Daily\Compare\Reports", exist_ok=True)
    os.makedirs(r"C:\Users\e201873842\Documents\Jupyter\Daily\Compare\sql\config", exist_ok=True)
except FileExistsError:
    # directory already exists
    pass

procpath = (r'C:\Users\e201873842\Documents\Jupyter\Daily\Compare\Process')
csvpath = (r'C:\Users\e201873842\Documents\Jupyter\Daily\Compare\Reports')
sqlpath = (r'C:\Users\e201873842\Documents\Jupyter\Daily\Compare\sql')
configpath = (r'C:\Users\e201873842\Documents\Jupyter\Daily\Compare\sql\config')
irange = (1,2,3,4,5,6,7,8,9,10,11,12)
jrange = (1,2,3)
for i in irange:
    with open(os.path.join(csvpath, 'compareReport{0}.csv').format(i), 'w') as fp: pass 
for j in jrange:
    with open(os.path.join(csvpath, 'compareReport1{0}.csv').format(i), 'w') as fp: pass 

krange = (0,1,2)
for k in krange:
    with open(os.path.join(sqlpath, 'sqlfile0{0}.sql').format(i), 'w') as fp: pass

    with open(os.path.join(procpath, 'dbListC.csv'), 'w') as fp: pass
    with open(os.path.join(procpath, 'dbListN.csv'), 'w') as fp: pass
    with open(os.path.join(procpath, 'tblListC.csv'), 'w') as fp: pass
    with open(os.path.join(procpath, 'tblListN.csv'), 'w') as fp: pass
    with open(os.path.join(procpath, 'schemaListC.csv'), 'w') as fp: pass
    with open(os.path.join(procpath, 'schemaListN.csv'), 'w') as fp: pass
    with open(os.path.join(procpath, 'collectionP1.csv'), 'w') as fp: pass
    with open(os.path.join(procpath, 'collectionQ1.csv'), 'w') as fp: pass

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-05 07:12:52


In [4]:
# Read excel file with core source tables
f1a = pd.read_csv(r'C:\Users\e201873842\Documents\Jupyter\Daily\Compare\sql\config\coreSource'+ csvtype +'.csv', encoding='unicode_escape')
f1b = pd.read_csv(r'C:\Users\e201873842\Documents\Jupyter\Daily\Compare\sql\config\noncoreSource'+ csvtype +'.csv', encoding='unicode_escape')

#Core
f1c = f1a['DatabaseName']           # (Core) Create dataframe with what is found in f1 dataframe
f1d = f1a['TableName']              # (Core) Create new dataframe with tables found in f1 dataframe
f1g = f1a['SchemaName']             # (Core) Create new dataframe with tables found in f1 dataframe

#NONCore
f1e = f1b['DatabaseName']           # (Non) Create dataframe with what is found in f1 dataframe
f1f = f1b['TableName']              # (Non) Create new dataframe with tables found in f1 dataframe
f1h = f1b['SchemaName']             # (Core) Create new dataframe with tables found in f1 dataframe

#Core
dbListC = f1c.drop_duplicates()     # Create dataframe without dupes and use in connection for-loop
tblListC = f1d.drop_duplicates()    # Create dataframe without dupes and use in connection string for-loop
schemaListC = f1g.drop_duplicates() # Create dataframe without dupes and use in connection string for-loop

#NONCore
dbListN = f1e.drop_duplicates()     # Create dataframe without dupes and use in connection for-loop
tblListN = f1f.drop_duplicates()    # Create dataframe without dupes and use in connection string for-loop
schemaListN = f1h.drop_duplicates() # Create dataframe without dupes and use in connection string for-loop

################################################################################
#Debug-Back up the *dbList, *tblList, *schemaList dataframes from input file for debug
################################################################################
#dbList files
with open(os.path.join(procpath, file00), 'a') as f:
    dbListC.to_csv(f, header=None, index=False, line_terminator='\n')
with open(os.path.join(procpath, file01), 'a') as f:
    dbListN.to_csv(f, header=None, index=False, line_terminator='\n')
#tblList files
with open(os.path.join(procpath, file02), 'a') as f:
    tblListC.to_csv(f, header=None, index=False, line_terminator='\n')
with open(os.path.join(procpath, file03), 'a') as f:
    tblListN.to_csv(f, header=None, index=False, line_terminator='\n')
#schemaList files
with open(os.path.join(procpath, file04), 'a') as f:
    schemaListC.to_csv(f, header=None, index=False, line_terminator='\n')
with open(os.path.join(procpath, file05), 'a') as f:
    schemaListN.to_csv(f, header=None, index=False, line_terminator='\n')

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-05 07:12:53


In [5]:
for x in dbListC:
    try:
        conn = pyodbc.connect('Server=PRODODSSQL;'
                              'Trusted_Connection=yes;'
                              'DRIVER={{SQL Server}};'
                              'Database={0}'.format(x))

        sql_query = pd.read_sql_query(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, TABLE_CATALOG as DatabaseName, TABLE_SCHEMA as SchemaName, TABLE_NAME as TableName, COLUMN_NAME as ColumnName, ORDINAL_POSITION as OrdinalPosition, IS_NULLABLE as IsNullable, DATA_TYPE as DataType
FROM INFORMATION_SCHEMA.COLUMNS
ORDER BY TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, ORDINAL_POSITION;
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from above

        dfp = pd.DataFrame(sql_query)
        with open(r'C:\Users\e201873842\Documents\Jupyter\Daily\Compare\Process\collectionP1.csv', 'a') as f:
            dfp.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')

        conn.close()
    except:
        continue
    finally:
        print('Production CORE collected: ', x, now01.strftime("%Y-%m-%d %H:%M:%S"))
        continue

now01 = datetime.datetime.now()
print('Production CORE Collection Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))

for x in dbListC:
    try:
        conn = pyodbc.connect('Server=QAODSSQL;'
                              'Trusted_Connection=yes;'
                              'DRIVER={{SQL Server}};'
                              'Database={0}'.format(x))

        sql_query = pd.read_sql_query(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, TABLE_CATALOG as DatabaseName, TABLE_SCHEMA as SchemaName, TABLE_NAME as TableName, COLUMN_NAME as ColumnName, ORDINAL_POSITION as OrdinalPosition, IS_NULLABLE as IsNullable, DATA_TYPE as DataType
FROM INFORMATION_SCHEMA.COLUMNS
ORDER BY TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, ORDINAL_POSITION;
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from above

        dfq = pd.DataFrame(sql_query)
        with open(r'C:\Users\e201873842\Documents\Jupyter\Daily\Compare\Process\collectionQ1.csv', 'a') as f:
            dfq.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')

        conn.close()
    except:
        continue
    finally:
        print('QA CORE collected: ', x, now01.strftime("%Y-%m-%d %H:%M:%S"))
        continue

now01 = datetime.datetime.now()
print('QA CORE Collection Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))

Production CORE collected:  ApplicationData 2021-05-05 07:12:53
Production CORE collected:  Assessment 2021-05-05 07:12:53
Production CORE collected:  GSDR 2021-05-05 07:12:53
Production CORE Collection Process Complete:  2021-05-05 07:12:56
QA CORE collected:  ApplicationData 2021-05-05 07:12:56
QA CORE collected:  Assessment 2021-05-05 07:12:56
QA CORE collected:  GSDR 2021-05-05 07:12:56
QA CORE Collection Process Complete:  2021-05-05 07:13:00


#LOAD excel collections into sp and sq dataframes for debug and process evaluations

In [6]:
df3 = pd.merge(dfp, dfq, how='outer', indicator='Exist')
df3 = df3.loc[df3['Exist'] != 'both']

In [7]:
with open(r'C:\Users\e201873842\Documents\Jupyter\Daily\Compare\Reports\CompareReport1.csv', 'a') as f:
    df3.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')

In [8]:
df4 = pd.merge(dfp, dfq, how='outer', indicator='Exist')
df4 = df4.loc[df4['Exist'] == 'both']

In [9]:
with open(r'C:\Users\e201873842\Documents\Jupyter\Daily\Compare\Reports\CompareReport2.csv', 'a') as f:
    df4.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')

In [10]:
# LOAD Daily results into Compare Tables (Difference & Match Exists) 

In [11]:
# Import CSV
data = pd.read_csv (r'C:\Users\e201873842\Documents\Jupyter\Daily\Compare\Reports\CompareReport1.csv', encoding='unicode_escape')   
df = pd.DataFrame(data, columns= ['TodaysDate','DatabaseName','SchemaName','TableName','ColumnName','OrdinalPosition','IsNullable','DataType','Exist'])
df2 = df.fillna(value=0)

df2.head()

,TodaysDate,DatabaseName,SchemaName,TableName,ColumnName,OrdinalPosition,IsNullable,DataType,Exist
0,20210505,GSDR,BUFORD,SDR_STU_C,SECTION504,123,NO,varchar,left_only
1,20210505,GSDR,BUFORD,SDR_STU_C,BI_LIT_DIPLOMA_SEAL,124,NO,varchar,left_only
2,20210505,GSDR,BUFORD,SDR_STU_C,CIVIC_ENGAGEMENT_DIPLOMA_SEAL,125,NO,varchar,left_only
3,20210505,GSDR,BUFORD,SDR_STU_C,CTAE_EMPLOYMENT_DIPLOMA_SEAL,126,NO,varchar,left_only
4,20210505,GSDR,BUFORD,SDR_STU_C,CTAE_PATHWAY_DIPLOMA_SEAL,127,NO,varchar,left_only


In [12]:
df2.columns.values

array(['TodaysDate', 'DatabaseName', 'SchemaName', 'TableName',
       'ColumnName', 'OrdinalPosition', 'IsNullable', 'DataType', 'Exist'],
      dtype=object)

In [13]:
df2.columns

Index(['TodaysDate', 'DatabaseName', 'SchemaName', 'TableName', 'ColumnName',
       'OrdinalPosition', 'IsNullable', 'DataType', 'Exist'],
      dtype='object')

In [14]:
# Import CSV
data = pd.read_csv (r'C:\Users\e201873842\Documents\Jupyter\Daily\Compare\Reports\CompareReport1.csv', encoding='unicode_escape')   
df = pd.DataFrame(data, columns= ['TodaysDate','DatabaseName','SchemaName','TableName','ColumnName','OrdinalPosition','IsNullable','DataType','Exist'])
df2 = df.fillna(value=0)

# Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

# Insert DataFrame to Table
for row in df2.itertuples():
    cursor.execute('''
                INSERT INTO Greg.dbo.CompareDifference (TodaysDate,DatabaseName,SchemaName,TableName,ColumnName,OrdinalPosition,IsNullable,DataType,Exist)
                VALUES (?,?,?,?,?,?,?,?,?);
                ''',
                row.TodaysDate,
                row.DatabaseName,
                row.SchemaName,
                row.TableName,
                row.ColumnName,
                row.OrdinalPosition,
                row.IsNullable,
                row.DataType,
                row.Exist
                )
conn.commit()

cursor.close()

print(' *** Note: csv file written to database table complete ***')

sql_query = pd.read_sql_query(''' 
select count(*) from Greg.dbo.CompareDifference
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from above
conn.close()
print(' *** Note: count from table data loaded is: ***', sql_query)
now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note: csv file written to database table complete ***
 *** Note: count from table data loaded is: ***         
0  56835
Processing Complete:  2021-05-05 07:13:05


In [15]:
# Import CSV
data = pd.read_csv (r'C:\Users\e201873842\Documents\Jupyter\Daily\Compare\Reports\CompareReport2.csv', encoding='unicode_escape')   
df = pd.DataFrame(data, columns= ['TodaysDate','DatabaseName','SchemaName','TableName','ColumnName','OrdinalPosition','IsNullable','DataType','Exist'])
df2 = df.fillna(value=0)

df2.head()

,TodaysDate,DatabaseName,SchemaName,TableName,ColumnName,OrdinalPosition,IsNullable,DataType,Exist
0,20210505,GSDR,BUFORD,SDR_STU_C,SCHOOL_YEAR,1,NO,smallint,both
1,20210505,GSDR,BUFORD,SDR_STU_C,PERIOD,2,NO,char,both
2,20210505,GSDR,BUFORD,SDR_STU_C,SYSTEM_CD,3,NO,char,both
3,20210505,GSDR,BUFORD,SDR_STU_C,SCHOOL_CD,4,NO,char,both
4,20210505,GSDR,BUFORD,SDR_STU_C,STUDENT_ID,5,NO,int,both


In [16]:
# Import CSV
data = pd.read_csv (r'C:\Users\e201873842\Documents\Jupyter\Daily\Compare\Reports\CompareReport2.csv', encoding='unicode_escape')   
df = pd.DataFrame(data, columns= ['TodaysDate','DatabaseName','SchemaName','TableName','ColumnName','OrdinalPosition','IsNullable','DataType','Exist'])
df2 = df.fillna(value=0)

# Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

# Insert DataFrame to Table
for row in df2.itertuples():
    cursor.execute('''
                INSERT INTO Greg.dbo.CompareMatchExists (TodaysDate,DatabaseName,SchemaName,TableName,ColumnName,OrdinalPosition,IsNullable,DataType,Exist)
                VALUES (?,?,?,?,?,?,?,?,?);
                ''',
                row.TodaysDate,
                row.DatabaseName,
                row.SchemaName,
                row.TableName,
                row.ColumnName,
                row.OrdinalPosition,
                row.IsNullable,
                row.DataType,
                row.Exist
                )
conn.commit()

cursor.close()

print(' *** Note: csv file written to database table complete ***')

sql_query = pd.read_sql_query(''' 
select count(*) from Greg.dbo.CompareMatchExists
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from above
conn.close()
print(' *** Note: count from table data loaded is: ***', sql_query)
now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note: csv file written to database table complete ***
 *** Note: count from table data loaded is: ***          
0  532270
Processing Complete:  2021-05-05 07:13:47
